In [1]:
import torch
import numpy as np
import torch.nn as nn
from click.termui import hidden_prompt_func
from openpyxl.styles.builtins import output

In [57]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_layer,output_size):
        super(RNN,self).__init__()
        self.i2h = nn.Linear(input_size,hidden_layer)
        self.w2h = nn.Linear(hidden_layer,hidden_layer)
        self.h2o = nn.Linear(hidden_layer,output_size)
        self.hidden_size = hidden_layer
        #hidden = self.init_hidden()
        
    def forward(self,x,hidden=None):
        if(hidden == None):
            hidden = self.init_hidden()
        x = self.i2h(x)
        hidden = self.w2h(hidden)
        hidden = torch.tanh(x + hidden)
        output = self.h2o(hidden)
        return output,hidden
    
    def init_hidden(self):
        return torch.zeros(1,self.hidden_size)
    

In [59]:
#test
a = torch.zeros(128,10)

model = RNN(10,30,20)
#model.init_hidden()
output,hidden = model(a)
output.shape

#test using torch.rnn
model = nn.RNN(input_size=10,hidden_size=30)
model(a)[0].shape

torch.Size([128, 30])

In [60]:
#writing rnn model
class RNNModel(nn.Module):
    def __init__(self,input_size,hidden_layer,output_size):
        super(RNNModel,self).__init__()
        self.rnncell = RNN(input_size,hidden_layer,output_size)
        
    def forward(self,x):
        len,batch_size,__ = x.size() # suppose x dim (seq_len, batch, input_size)
        outputs = []
        for i in range(len):
            output, hidden = self.rnncell(x[i])
            outputs.append(output)
        outputs = torch.stack(outputs,dim=0)
        return outputs,hidden     

In [61]:
#test
b = torch.zeros(10,128,5)
model = RNNModel(5,20,15)
model(b)[0].shape

torch.Size([10, 128, 15])